In [1]:
from quchem.Simulating_Quantum_Circuit import *
from quchem.Ansatz_Generator_Functions import *
from openfermion.ops import QubitOperator
HF_circ = [cirq.X.on(cirq.LineQubit(0)), cirq.X.on(cirq.LineQubit(1))]
x = QubitOperator('X0 Y1 Z2 Y3', 0.25j)
theta = np.pi
full_exp_circ_obj = full_exponentiated_PauliWord_circuit(x, theta)
UCCSD_circ = cirq.Circuit(cirq.decompose_once((full_exp_circ_obj(*cirq.LineQubit.range(full_exp_circ_obj.num_qubits())))))
YY = QubitOperator('X0 X1 Y3', 0.25j)
pauliword_change_and_measure_obj = change_pauliword_to_Z_basis_then_measure(YY)
P_measure = cirq.Circuit(cirq.decompose_once((pauliword_change_and_measure_obj(*cirq.LineQubit.range(pauliword_change_and_measure_obj.num_qubits())))))

full_circuit = cirq.Circuit([*HF_circ, *UCCSD_circ.all_operations(), *P_measure.all_operations()])
print(full_circuit)

0: ───X──────────H──────────@───────────────────────────────────────@──────────H───────────Ry(-0.5π)───M───
                            │                                       │                                  │
1: ───X──────────Rx(0.5π)───X───@───────────────────────@───────────X──────────Rx(-0.5π)───Ry(-0.5π)───M───
                                │                       │                                              │
2: ─────────────────────────────X───@───────────────@───X──────────────────────────────────────────────┼───
                                    │               │                                                  │
3: ───Rx(0.5π)──────────────────────X───Rz(-0.5π)───X───Rx(-0.5π)───Rx(0.5π)───────────────────────────M───


## Get_Histogram_key

In [2]:
YY = QubitOperator('X0 X1 Y3', 0.25j)
Get_Histogram_key(YY)

'0,1,3'

## Simulate_Quantum_Circuit

In [3]:
num_shots = 1000
YY = QubitOperator('X0 X1 Y3', 0.25j)

histogram_string= Get_Histogram_key(YY)
Simulate_Quantum_Circuit(full_circuit, num_shots, histogram_string)

Counter({0: 256, 5: 244, 6: 245, 3: 255})

## Get_wavefunction

In [4]:
YY = QubitOperator('X0 X1 Y3', 0.25j)

cirq_NO_M = cirq.Circuit([*HF_circ, *UCCSD_circ.all_operations()])

histogram_string= Get_Histogram_key(YY)
Get_wavefunction(cirq_NO_M, sig_figs=3)

array([[ 0.   +0.j   ],
       [ 0.   +0.707j],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [-0.   +0.j   ],
       [-0.   +0.j   ],
       [-0.   +0.j   ],
       [ 0.707+0.j   ],
       [-0.   +0.j   ],
       [-0.   +0.j   ],
       [-0.   +0.j   ]], dtype=complex64)

## Return_as_binary

In [5]:
num_shots = 1000
YY = QubitOperator('X0 X1 Y3', 0.25j)

histogram_string= Get_Histogram_key(YY)
c_result = Simulate_Quantum_Circuit(full_circuit, num_shots, histogram_string)
Return_as_binary(c_result, histogram_string)

{'011': 224, '101': 268, '000': 266, '110': 242}

## expectation_value_by_parity

In [6]:
num_shots = 10000
YY = QubitOperator('X0 X1 Y3', 0.25j)

histogram_string= Get_Histogram_key(YY)
c_result = Simulate_Quantum_Circuit(full_circuit, num_shots, histogram_string)
b_result = Return_as_binary(c_result, histogram_string)

expectation_value_by_parity(b_result)

1.0

In [7]:
circuit = cirq.Circuit(cirq.H(cirq.LineQubit(1)),
                       cirq.CNOT(cirq.LineQubit(1),cirq.LineQubit(2)),
                       cirq.measure(cirq.LineQubit(1),cirq.LineQubit(2)))
print(circuit)

print('')
print('state = (1/2^0.5)|00〉+ (1/2^0.5)|11〉')

simulator = cirq.Simulator()
raw_result = simulator.run(circuit, repetitions=1000)
hist_result = raw_result.histogram(key='1,2')
print('')
#print(hist_result)
bin_dict = Return_as_binary(hist_result, '1,2')
print(bin_dict)

print('')
print('<ZZ> = ', expectation_value_by_parity(bin_dict))
#Get_wavefunction(circuit, sig_figs=3)

1: ───H───@───M───
          │   │
2: ───────X───M───

state = (1/2^0.5)|00〉+ (1/2^0.5)|11〉

{'11': 488, '00': 512}

<ZZ> =  1.0


In [8]:
circuit = cirq.Circuit(cirq.H(cirq.LineQubit(1)),
                       cirq.CNOT(cirq.LineQubit(1),cirq.LineQubit(2)),
                       cirq.CNOT(cirq.LineQubit(2),cirq.LineQubit(1)),
                       cirq.measure(cirq.LineQubit(1),cirq.LineQubit(2)))
print(circuit)

print('')
print('state = (1/2^0.5)|00〉+ (1/2^0.5)|01〉')

simulator = cirq.Simulator()
raw_result = simulator.run(circuit, repetitions=1000)
hist_result = raw_result.histogram(key='1,2')
print('')
#print(hist_result)
bin_dict = Return_as_binary(hist_result, '1,2')
print(bin_dict)

print('')
print('<ZZ> = ', expectation_value_by_parity(bin_dict))
#Get_wavefunction(circuit, sig_figs=3)

1: ───H───@───X───M───
          │   │   │
2: ───────X───@───M───

state = (1/2^0.5)|00〉+ (1/2^0.5)|01〉

{'00': 482, '01': 518}

<ZZ> =  -0.036


In [9]:
circuit = cirq.Circuit(cirq.H(cirq.LineQubit(1)),
                       cirq.H(cirq.LineQubit(2)),
                       cirq.X(cirq.LineQubit(3)),
                       cirq.measure(cirq.LineQubit(1),cirq.LineQubit(2),cirq.LineQubit(3)))
print(circuit)

print('')
print('state = (1/2)|001〉+ (1/2)|011〉+ (1/2)|101〉+ (1/2)|111〉')

simulator = cirq.Simulator()
raw_result = simulator.run(circuit, repetitions=100000)
hist_result = raw_result.histogram(key='1,2,3')
print('')
#print(hist_result)
bin_dict = Return_as_binary(hist_result, '1,2,3')
print(bin_dict)

print('')
print('<ZZ> = ', expectation_value_by_parity(bin_dict))
#Get_wavefunction(circuit, sig_figs=3)

1: ───H───M───
          │
2: ───H───M───
          │
3: ───X───M───

state = (1/2)|001〉+ (1/2)|011〉+ (1/2)|101〉+ (1/2)|111〉

{'111': 24795, '001': 25127, '011': 25022, '101': 25056}

<ZZ> =  0.00156


In [ ]:
# # wrong method!

# def expectation_value_by_parityWRONG(binary_counter_result, constant):
#     """

#     < Z >  = (num_0 - num_1) / total_num_measurements = (num_0 - num_1) / (num_0 + num_1)

#     note that for multiple qubits one multiplies <Z> on each line. Therefore can calculate value from parity
#     of output bit string

#     :param binary_counter_result:
#     :type binary_counter_result: dict
#     e.g.
#         {'000': 1295,
#          '101': 1216,
#          '100': 1270,
#          '110': 1247,
#          '001': 1263,
#          '011': 1211,
#          '010': 1226,
#          '111': 1272
#          }

#     :return: expectation value <Z>
#     :rtype: float
#     """
    
#     qubit_expectation_dict = {}
    
#     for state in binary_counter_result:
#         for qubit_index, bit in enumerate(state):
#             if qubit_index in qubit_expectation_dict:
#                 if int(bit) == 0:
#                     qubit_expectation_dict[qubit_index]['No_zeros']+= binary_counter_result[state]
#                 elif int(bit) == 1:
#                     qubit_expectation_dict[qubit_index]['No_ones']+= binary_counter_result[state]
#                 else:
#                     raise ValueError('{} is not a bit'.format(bit))
                    
#             else:
#                 qubit_expectation_dict[qubit_index]= {'No_zeros':0, 'No_ones':0}
                
#                 if int(bit) == 0:
#                     qubit_expectation_dict[qubit_index]['No_zeros']+= binary_counter_result[state]
#                 elif int(bit) == 1:
#                     qubit_expectation_dict[qubit_index]['No_ones']+= binary_counter_result[state]
#                 else:
#                     raise ValueError('{} is not a bit'.format(bit))
                    
                    
#     #print(qubit_expectation_dict)
#     exp_list=[]
#     for qubit_index in qubit_expectation_dict:
#         expectation_value = (qubit_expectation_dict[qubit_index]['No_zeros']-qubit_expectation_dict[qubit_index]['No_ones'])/(qubit_expectation_dict[qubit_index]['No_zeros']+qubit_expectation_dict[qubit_index]['No_ones'])
        
#         E_i = expectation_value * constant
#         exp_list.append(E_i)
#     print(exp_list)
#     return sum(exp_list)


# b_result_test = {'000': 1000,
#          '100': 1000,
#          }

# expectation_value_by_parityWRONG(b_result_test,0.5)

In [ ]:
# wrong method!
from functools import reduce 
def expectation_value_by_parityWRONG(binary_counter_result):
    """

    < Z >  = (num_0 - num_1) / total_num_measurements = (num_0 - num_1) / (num_0 + num_1)

    note that for multiple qubits one multiplies <Z> on each line. Therefore can calculate value from parity
    of output bit string

    :param binary_counter_result:
    :type binary_counter_result: dict
    e.g.
        {'000': 1295,
         '101': 1216,
         '100': 1270,
         '110': 1247,
         '001': 1263,
         '011': 1211,
         '010': 1226,
         '111': 1272
         }

    :return: expectation value <Z>
    :rtype: float
    """
    
    qubit_expectation_dict = {}
    
    for state in binary_counter_result:
        for qubit_index, bit in enumerate(state):
            if qubit_index in qubit_expectation_dict:
                if int(bit) == 0:
                    qubit_expectation_dict[qubit_index]['No_zeros']+= binary_counter_result[state]
                elif int(bit) == 1:
                    qubit_expectation_dict[qubit_index]['No_ones']+= binary_counter_result[state]
                else:
                    raise ValueError('{} is not a bit'.format(bit))
                    
            else:
                qubit_expectation_dict[qubit_index]= {'No_zeros':0, 'No_ones':0}
                
                if int(bit) == 0:
                    qubit_expectation_dict[qubit_index]['No_zeros']+= binary_counter_result[state]
                elif int(bit) == 1:
                    qubit_expectation_dict[qubit_index]['No_ones']+= binary_counter_result[state]
                else:
                    raise ValueError('{} is not a bit'.format(bit))
                    
                    
    #print(qubit_expectation_dict)
    exp_list=[]
    for qubit_index in qubit_expectation_dict:
        expectation_value = (qubit_expectation_dict[qubit_index]['No_zeros']-qubit_expectation_dict[qubit_index]['No_ones'])/(qubit_expectation_dict[qubit_index]['No_zeros']+qubit_expectation_dict[qubit_index]['No_ones'])
        
        E_i = expectation_value
        exp_list.append(E_i)
    print(exp_list)
    return reduce(lambda x, y: x * y, exp_list)


b_result_test = {'000': 1000,
         '010': 800,
         }

expectation_value_by_parityWRONG(b_result_test)

In [ ]:
# class VQE_Experiment():
#     def __init__(self, qubitHamiltonian, ansatz_circuit, n_shots):
#         self.qubitHamiltonian = qubitHamiltonian
#         self.ansatz_circuit = ansatz_circuit
#         self.n_shots = n_shots

#     def Calc_Energy(self):

#         E_list=[]
#         for qubitOp in self.qubitHamiltonian:
#             for PauliWord, const in qubitOp.terms.items():
#                 if PauliWord is not ():
#                     Q_circuit = Generate_Full_Q_Circuit(self.ansatz_circuit, qubitOp)
#                     hist_key_str = Get_Histogram_key(qubitOp)
#                     int_state_counter = Simulate_Quantum_Circuit(Q_circuit, self.n_shots, hist_key_str)
#                     binary_state_counter = Return_as_binary(int_state_counter, hist_key_str)
#                     exp_result = expectation_value_by_parityWRONG(binary_state_counter,const) #CHANGE HERE
#                     E_list.append(exp_result)
                    
# #                     print(binary_state_counter)
# #                     print(const)
                    
# #                     print(exp_result)
# #                     print('###')
# #                     print('')
                    
#                 else:
#                     E_list.append(const)
#         return sum(E_list).real

#     def Get_wavefunction_of_state(self, sig_figs=3):
#         return Get_wavefunction(self.ansatz_circuit, sig_figs=sig_figs)

In [10]:
from quchem.quantum_circuit_functions import Generate_Full_Q_Circuit
class VQE_Experiment():
    def __init__(self, qubitHamiltonian, ansatz_circuit, n_shots):
        self.qubitHamiltonian = qubitHamiltonian
        self.ansatz_circuit = ansatz_circuit
        self.n_shots = n_shots

    def Calc_Energy(self):

        E_list=[]
        for qubitOp in self.qubitHamiltonian:
            for PauliWord, const in qubitOp.terms.items():
                if PauliWord is not ():
                    Q_circuit = Generate_Full_Q_Circuit(self.ansatz_circuit, qubitOp)
                    hist_key_str = Get_Histogram_key(qubitOp)
                    int_state_counter = Simulate_Quantum_Circuit(Q_circuit, self.n_shots, hist_key_str)
                    binary_state_counter = Return_as_binary(int_state_counter, hist_key_str)
                    exp_result = expectation_value_by_parity(binary_state_counter)
                    
                    print(qubitOp)
                    term_to_add = (exp_result*const)
                    print(exp_result, const, '==>', term_to_add)
                    print('')
                    E_list.append(term_to_add)
                else:
                    E_list.append(const)
        return sum(E_list).real

    def Get_wavefunction_of_state(self, sig_figs=3):
        return Get_wavefunction(self.ansatz_circuit, sig_figs=sig_figs)



|ΨH2g〉= 0.9939|0011〉−0.1106|1100〉

<Z0> = (0.9939**2)*1+(0.1106**2)*-1

In [ ]:
(0.9939**2)*1+(0.1106**2)*-1

In [ ]:
Coefficient_list = [1/np.sqrt(2), 0, 1/np.sqrt(2), 0]
num_qub = 2
alpha_j = Get_state_prep_dict(num_qub, Coefficient_list=Coefficient_list)
state_circ = State_Prep_Circuit(alpha_j)
circuit = (cirq.Circuit(cirq.decompose_once((state_circ(*cirq.LineQubit.range(state_circ.num_qubits()))))))
VQE_exp = VQE_Experiment(QubitHam, circuit, n_shots)
VQE_exp.Get_wavefunction_of_state()
alpha_j

In [ ]:
THETA=np.pi/2
n_shots=10000

ansatz_cirq_circuit = H2_ansatz(THETA)
VQE_exp = VQE_Experiment(QubitHam, circuit, n_shots)
VQE_exp.Calc_Energy()

In [ ]:
THETA=np.pi/2
n_shots=10000

ansatz_cirq_circuit = H2_ansatz(THETA)
VQE_exp = VQE_Experiment(QubitHam, ansatz_cirq_circuit, n_shots)
VQE_exp.Calc_Energy()

In [ ]:

n_shots=1000

def GIVE_ENERGY(THETA):
   
    ansatz_cirq_circuit = H2_ansatz(THETA)

    VQE_exp = VQE_Experiment(QubitHam, ansatz_cirq_circuit, n_shots)

    return VQE_exp.Calc_Energy()
### full angle scan

import matplotlib.pyplot as plt
%matplotlib inline

theta_list = np.arange(0,2*np.pi, 0.1)

E_list = [GIVE_ENERGY(theta) for theta in theta_list]

plt.plot(E_list)
print(min(E_list))

In [11]:
from quchem.Hamiltonian_Generator_Functions import *
### Parameters
Molecule = 'H2'
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., 0.74))]
basis = 'sto-3g'

### Get Hamiltonian
Hamilt = Hamiltonian(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis=basis,
                     multiplicity=1,
                     geometry=geometry)  # normally None!

Hamilt.Get_Molecular_Hamiltonian(Get_H_matrix=False)
QubitHam = Hamilt.Get_Qubit_Hamiltonian(transformation='JW')

ansatz_obj = Ansatz(Hamilt.molecule.n_electrons, Hamilt.molecule.n_qubits)

Sec_Quant_CC_ia_ops, Sec_Quant_CC_ijab_ops, theta_parameters_ia, theta_parameters_ijab = ansatz_obj.Get_ia_and_ijab_terms()

Qubit_Op_list_Second_Quant_CC_Ops_ia, Qubit_Op_list_Second_Quant_CC_Ops_ijab = ansatz_obj.UCCSD_single_trotter_step(Sec_Quant_CC_ia_ops, Sec_Quant_CC_ijab_ops,
                                                                                                                    transformation='JW')

full_ansatz_Q_Circ = Ansatz_Circuit(Qubit_Op_list_Second_Quant_CC_Ops_ia, Qubit_Op_list_Second_Quant_CC_Ops_ijab,
             Hamilt.molecule.n_qubits, Hamilt.molecule.n_electrons)

ansatz_cirq_circuit = full_ansatz_Q_Circ.Get_Full_HF_UCCSD_QC(theta_parameters_ia, theta_parameters_ijab)

In [ ]:
QubitHam

$$\begin{aligned} H &=h_{0} I+h_{1} Z_{0}+h_{2} Z_{1}+h_{3} Z_{2}+h_{4} Z_{3} \\ &+h_{5} Z_{0} Z_{1}+h_{6} Z_{0} Z_{2}+h_{7} Z_{1} Z_{2}+h_{8} Z_{0} Z_{3}+h_{9} Z_{1} Z_{3} \\ &+h_{10} Z_{2} Z_{3}+h_{11} Y_{0} Y_{1} X_{2} X_{3}+h_{12} X_{0} Y_{1} Y_{2} X_{3} \\ &+h_{13} Y_{0} X_{1} X_{2} Y_{3}+h_{14} X_{0} X_{1} Y_{2} Y_{3} \end{aligned}$$

In [ ]:
Sec_Quant_CC_ijab_ops

In [ ]:
from openfermion.transforms import jordan_wigner

Second_Quant_CC_single_Trot_list_ijab=[]

for OP in Sec_Quant_CC_ijab_ops:
    JW_OP = jordan_wigner(OP)
    Second_Quant_CC_single_Trot_list_ijab.append(JW_OP)
Second_Quant_CC_single_Trot_list_ijab

In [ ]:
ansatz_obj.Get_JW_HF_state()

In [ ]:
from numpy import kron
import numpy as np
from functools import reduce
zero = np.array([[1], [0]])
one = np.array([[0], [1]])
STATE=[one,one,zero,zero]
reduce(kron, STATE)

In [ ]:
HF_reference_ket= reduce(kron, STATE)


from openfermion.transforms import get_sparse_operator
UCCSD_QubitOp = QubitOperator('Y0 X1 X2 X3', -1j)
UCCSD_operator = get_sparse_operator(UCCSD_QubitOp)

In [ ]:
pauliDict={'X':np.array([[0,1],[1,0]]),
          'Y':np.array([[0,-1j],[1j,0]]),
          'Z':np.array([[1,0],[0,-1]]),
          'I': np.eye(2)}
pauliDict

In [ ]:
list_of_ops = [pauliDict[term[0]] for term in 'Y0 X1 X2 X3'.split(' ')]
matrix = reduce(kron, list_of_ops) *-1j

np.array_equal(UCCSD_operator.todense(), matrix)

In [ ]:
theta=np.pi/2
ansatz_state_ket = scipy.sparse.linalg.expm_multiply(theta*UCCSD_operator, HF_reference_ket)
ansatz_state_ket

In [ ]:
pauliDict_convert={'X':cirq.ry(-np.pi/2)._unitary_(),
          'Y':cirq.rx(np.pi/2)._unitary_(),
          'Z':np.eye(2),
          'I': np.zeros([2,2])}
pauliDict_convert

In [ ]:
reference_ket= reduce(kron, STATE)
new_state = scipy.sparse.linalg.expm_multiply(generator, reference_ket)

In [ ]:
n_shots= 1000

VQE_exp = VQE_Experiment(QubitHam, ansatz_cirq_circuit, n_shots)
VQE_exp.Calc_Energy()

In [ ]:
n_shots=1000

def GIVE_ENERGY(theta_ia_theta_jab_list):
    theta_ia = theta_ia_theta_jab_list[:len(theta_parameters_ia)]
    theta_ijab = theta_ia_theta_jab_list[len(theta_parameters_ia):]
    
    ansatz_cirq_circuit = full_ansatz_Q_Circ.Get_Full_HF_UCCSD_QC(theta_parameters_ia, theta_parameters_ijab)

    VQE_exp = VQE_Experiment(QubitHam, ansatz_cirq_circuit, n_shots)
    

    return VQE_exp.Calc_Energy()

### optimizer
from quchem.Scipy_Optimizer import *
# THETA_params = [*theta_parameters_ia, *theta_parameters_ijab]
THETA_params=[1,2,3]
GG = Optimizer(GIVE_ENERGY, THETA_params, 'Nelder-Mead', store_values=True, display_iter_steps=True,
               tol=1e-5,
               display_convergence_message=True)
GG.get_env(50)
GG.plot_convergence()
plt.show()

In [ ]:
Qubit_Op_list_Second_Quant_CC_Ops_ia

In [ ]:
# from quchem.quantum_circuit_functions import Generate_Full_Q_Circuit


# class VQE_Experiment():
#     def __init__(self, qubitHamiltonian, ansatz_circuit, n_shots):
#         self.qubitHamiltonian = qubitHamiltonian
#         self.ansatz_circuit = ansatz_circuit
#         self.n_shots = n_shots

#     def Calc_Energy(self):
#         t_results=[]
#         E_list=[]
#         for qubitOp in self.qubitHamiltonian:
#             for PauliWord, const in qubitOp.terms.items():
#                 if PauliWord is not ():
                    
#                     Q_circuit = Generate_Full_Q_Circuit(self.ansatz_circuit, qubitOp)
#                     hist_key_str = Get_Histogram_key(qubitOp)
#                     int_state_counter = Simulate_Quantum_Circuit(Q_circuit, self.n_shots, hist_key_str)
#                     binary_state_counter = Return_as_binary(int_state_counter, hist_key_str)
#                     exp_result = expectation_value_by_parity(binary_state_counter)
#                     E_list.append(exp_result*const)
                    
#                     t_results.append((exp_result, const))
#                 else:
#                     E_list.append(const)
#                     t_results.append((1, const))
                    
                    
#         return sum(E_list).real, t_results

#     def Get_wavefunction_of_state(self, sig_figs=3):
#         return Get_wavefunction(self.ansatz_circuit, sig_figs=sig_figs)


In [ ]:
from quchem.Simulating_Quantum_Circuit import *
from quchem.Ansatz_Generator_Functions import *
from openfermion.ops import QubitOperator

def H2_ansatz(theta):
    HF_circ = [cirq.X.on(cirq.LineQubit(0)), cirq.X.on(cirq.LineQubit(1))]
    
    full_exp_circ_obj = full_exponentiated_PauliWord_circuit(QubitOperator('Y0 X1 X2 X3', -1j), theta)
    UCCSD_circ = cirq.Circuit(cirq.decompose_once((full_exp_circ_obj(*cirq.LineQubit.range(full_exp_circ_obj.num_qubits())))))
    full_circuit = cirq.Circuit([*HF_circ, *UCCSD_circ.all_operations()])
    
    return full_circuit
    
H2_ansatz(np.pi)               

In [ ]:
n_shots=1000

def GIVE_ENERGY(THETA):
   
    ansatz_cirq_circuit = H2_ansatz(THETA)

    VQE_exp = VQE_Experiment(QubitHam, ansatz_cirq_circuit, n_shots)

    return VQE_exp.Calc_Energy()


In [ ]:
### full angle scan

import matplotlib.pyplot as plt
%matplotlib inline

theta_list = np.arange(0,2*np.pi, 0.01)

E_list = [GIVE_ENERGY(theta) for theta in theta_list]

plt.plot(E_list)
print(min(E_list))

In [ ]:
## optimzer

from quchem.Scipy_Optimizer import *
THETA_params=[2]
GG = Optimizer(GIVE_ENERGY, THETA_params, 'Nelder-Mead', store_values=True, display_iter_steps=True,
               tol=1e-5,
               display_convergence_message=True)
GG.get_env(50)
GG.plot_convergence()
plt.show()

In [ ]:
def expectation_value_by_parityWRONG(binary_counter_result):
    """

    < Z >  = (num_0 - num_1) / total_num_measurements = (num_0 - num_1) / (num_0 + num_1)

    note that for multiple qubits one multiplies <Z> on each line. Therefore can calculate value from parity
    of output bit string

    :param binary_counter_result:
    :type binary_counter_result: dict
    e.g.
        {
            1: {'11': 10000},
            2: {'10': 9998, '01': 2},
            3: {'10': 10000},
            4: {'10': 10000},
            5: {'10': 10000},
            6: {'00': 9995, '01': 4, '10': 1},
            7: {'1': 9334, '0': 666},
            8: {'1': 9351, '0': 649},
            9: {'0': 9606, '1': 394},
            10: {'0': 9594, '1': 406}
        }

    :return: expectation value <Z>
    :rtype: float
    """
    n_0 =0
    n_1 =0
    for state in binary_counter_result:
        for bit in state:
            if int(bit) == 0:
                n_0 += binary_counter_result[state]
            elif int(bit) == 1:
                n_1 += binary_counter_result[state]
            else:
                raise ValueError('{} is not a bit'.format(bit))
            
    expectation_value = (n_0-n_1) / (n_0+n_1)
    return expectation_value

In [ ]:
class VQE_Experiment():
    def __init__(self, qubitHamiltonian, ansatz_circuit, n_shots):
        self.qubitHamiltonian = qubitHamiltonian
        self.ansatz_circuit = ansatz_circuit
        self.n_shots = n_shots

    def Calc_Energy(self):

        E_list=[]
        for qubitOp in self.qubitHamiltonian:
            for PauliWord, const in qubitOp.terms.items():
                if PauliWord is not ():
                    Q_circuit = Generate_Full_Q_Circuit(self.ansatz_circuit, qubitOp)
                    
                    print(Q_circuit)
                    print(qubitOp)
                    print('')
                    
                    hist_key_str = Get_Histogram_key(qubitOp)
                    int_state_counter = Simulate_Quantum_Circuit(Q_circuit, self.n_shots, hist_key_str)
                    binary_state_counter = Return_as_binary(int_state_counter, hist_key_str)
                    exp_result = expectation_value_by_parity(binary_state_counter)
                    print(exp_result)
                    E_list.append(exp_result*const)
                    
                    print(binary_state_counter)
                    
                    print('')
                    print('')
                    print('###')
                else:
                    E_list.append(const)
        print(E_list)
        return sum(E_list).real

    def Get_wavefunction_of_state(self, sig_figs=3):
        return Get_wavefunction(self.ansatz_circuit, sig_figs=sig_figs)


In [ ]:
ansatz = H2_ansatz(np.pi) 
xx = VQE_Experiment(QubitHam, ansatz, 1000)
xx.Calc_Energy()

In [ ]:
#TODO
# write maths of PauliMeasurement!
# something is off!!!

$$ U_{UCCSD}|HF\rangle = |\psi_{UCCSD}\rangle$$ 

$$ E =  \langle \psi_{UCCSD}| H |\psi_{UCCSD}\rangle$$ 

$$H = \sum_{i} \alpha_{i} P_{i}$$ 

$$ E_{i} =  \langle \psi_{UCCSD}| P_{i} |\psi_{UCCSD}\rangle$$ 

$$ E_{i} =  \langle \psi_{UCCSD}| P_{i} |\psi_{UCCSD}\rangle = M_{z} U_{i}|\psi_{UCCSD}\rangle$$ 

- where : $ U_{i}$ is a change of basis of $P_{i}$ to Z basis

AKA requires the following **single** qubit transforms:

$$X|\pm\rangle = ZH|\pm\rangle = Z|0/1\rangle$$ 

$$Y|i\pm \rangle = ZR_{x}\big(-\frac{\pi}{2} \big)|i\pm\rangle = Z|0/1\rangle$$ 

- note $Z$ here is a measurement in Z basis!

overall we are doing

$$ E_{i} =  P_{i} |\psi_{UCCSD}\rangle = M_{z} U_{i}|\psi_{UCCSD}\rangle = M_{z}|\psi_{P_{i}}\rangle = \langle\psi_{P_{i}}|M_{z}^{\dagger} M_{z}|\psi_{P_{i}}\rangle $$ 

$$P|\psi_{UCCSD}\rangle$$ 

$$X|\pm\rangle = ZH|\pm\rangle = Z|0/1\rangle$$ 

$$Y|i\pm \rangle = ZR_{x}\big(\frac{\pi}{2} \big)|i\pm\rangle = Z|0/1\rangle$$ 

In [ ]:
circuit = cirq.Circuit(cirq.H(cirq.LineQubit(1)),cirq.X(cirq.LineQubit(1)), cirq.measure(cirq.LineQubit(1)))

simulator = cirq.Simulator()
raw_result = simulator.run(circuit, repetitions=100)
hist_result = raw_result.histogram(key='1')
hist_result

In [ ]:
circuit = cirq.Circuit(cirq.H(cirq.LineQubit(1)),
                       cirq.CNOT(cirq.LineQubit(1),cirq.LineQubit(2)),
                       cirq.measure(cirq.LineQubit(1),cirq.LineQubit(2)))
simulator = cirq.Simulator()
raw_result = simulator.run(circuit, repetitions=100)
hist_result = raw_result.histogram(key='1,2')
print(hist_result)

bin_dict = Return_as_binary(hist_result, '1,2')

print(bin_dict)

print(expectation_value_by_parity(bin_dict))

Get_wavefunction(circuit, sig_figs=3)

In [ ]:
circuit = cirq.Circuit(cirq.H(cirq.LineQubit(1)),
                       cirq.X(cirq.LineQubit(1)),
                       cirq.measure(cirq.LineQubit(1),cirq.LineQubit(2)))
simulator = cirq.Simulator()
raw_result = simulator.run(circuit, repetitions=1000)
hist_result = raw_result.histogram(key='1,2')
print(hist_result)

bin_dict = Return_as_binary(hist_result, '1,2')

print(bin_dict)

print(expectation_value_by_parity(bin_dict))

Get_wavefunction(circuit, sig_figs=3)

In [ ]:
((526+474*2)-526)/((526+474*2)+526)

In [ ]:
(474-526)/(474+526)

In [ ]:
QubitHam

In [ ]:
P_words =[]
P_Nos=[]
for term in [list(op.terms.keys())[0] for op in QubitHam]:
    if term:
        P_word=[]
        N_list=[]
        for N, P in term:
            P_word.append(P)
            N_list.append(N)
        P_Nos.append(N_list)
        P_words.append(P_word)

print(P_Nos)
P_words

In [ ]:
P_Nos

In [ ]:
P_words= [ ['Z', 'I', 'I', 'I'],
            ['I', 'Z', 'I', 'I'],
            ['I', 'I', 'Z', 'I'],
            ['I', 'I', 'I', 'Z'],
            ['Z', 'Z', 'I', 'I'],
            ['Y', 'X', 'X', 'Y'],
            ['Y', 'Y', 'X', 'X'],
            ['X', 'X', 'Y', 'Y'],
            ['X', 'Y', 'Y', 'X'],
            ['Z', 'I', 'Z', 'I'],
            ['Z', 'I', 'I', 'Z'],
            ['I', 'Z', 'Z', 'I'],
            ['I', 'Z', 'I', 'Z'],
            ['I', 'I', 'Z', 'Z']]

In [ ]:
pauliDict_convert={'X':cirq.ry(-np.pi/2)._unitary_(),
          'Y':cirq.rx(np.pi/2)._unitary_(),
          'Z':np.eye(2),
          'I': np.zeros([2,2])}
pauliDict_convert

In [ ]:
sig_figs =5
theta = 3
CONSTANT=100
i=8

### ansatz
anastz_circ = H2_ansatz(theta)    
simulator = cirq.Simulator()
result = simulator.simulate(anastz_circ, qubit_order=anastz_circ.all_qubits())
ket_ansatz_state = np.around(result.final_state, sig_figs)

bra_ansatz_state = ket_ansatz_state.conj()


### term to measure

print(P_words[i])
from numpy import kron
from functools import reduce
operator = reduce(kron,[pauliDict_convert[P] for P in P_words[i]])


## energy
energy = bra_ansatz_state.dot(operator.dot(ket_ansatz_state))*CONSTANT
energy

In [ ]:
sig_figs =5
theta = 3

### ansatz
anastz_circ = H2_ansatz(theta)    
simulator = cirq.Simulator()
result = simulator.simulate(anastz_circ, qubit_order=anastz_circ.all_qubits())
ket_ansatz_state = np.around(result.final_state, sig_figs)

bra_ansatz_state = ket_ansatz_state.conj()

E_list=[]

consts = [list(op.terms.values())[0] for op in QubitHam]
for const in consts[1:]:
    operator = reduce(kron,[pauliDict_convert[P] for P in P_words[i]])
    energy = bra_ansatz_state.dot(operator.dot(ket_ansatz_state))*const
    E_list.append(energy.real)
    print(energy.real)
E_list
sum(E_list)+[consts[0]]

In [ ]:
circuit = cirq.Circuit(cirq.H(cirq.LineQubit(1)),
                       cirq.CNOT(cirq.LineQubit(1),cirq.LineQubit(2)),
                       cirq.measure(cirq.LineQubit(2)),
                      cirq.X(cirq.LineQubit(1)))

print(circuit)

simulator = cirq.Simulator()
raw_result = simulator.run(circuit, repetitions=1000)
hist_result = raw_result.histogram(key='2')
print(hist_result)

In [ ]:
class VQE_Experiment():
    def __init__(self, qubitHamiltonian, ansatz_circuit, n_shots):
        self.qubitHamiltonian = qubitHamiltonian
        self.ansatz_circuit = ansatz_circuit
        self.n_shots = n_shots

    def Calc_Energy(self):

        E_list=[]
        for qubitOp in self.qubitHamiltonian:
            for PauliWord, const in qubitOp.terms.items():
                if PauliWord is not ():
                    Q_circuit = Generate_Full_Q_Circuit(self.ansatz_circuit, qubitOp)
                    
                    print(Q_circuit)
                    print(qubitOp)
                    print('')
                    
                    hist_key_str = Get_Histogram_key(qubitOp)
                    int_state_counter = Simulate_Quantum_Circuit(Q_circuit, self.n_shots, hist_key_str)
                    binary_state_counter = Return_as_binary(int_state_counter, hist_key_str)
                    exp_result = expectation_value_by_parity(binary_state_counter)
                    print(exp_result)
                    E_list.append(exp_result*const)
                    
                    print(binary_state_counter)
                    
                    print('')
                    print('')
                    print('###')
                else:
                    E_list.append(const)
        print(E_list)
        return sum(E_list).real

    def Get_wavefunction_of_state(self, sig_figs=3):
        return Get_wavefunction(self.ansatz_circuit, sig_figs=sig_figs)


In [ ]:
from quchem.quantum_circuit_functions import *
# |Ψ H2〉= 0.9939|0011〉−0.1106|1100〉
# |0011〉 = |3〉
# |1100〉= |12〉

coef = [0., 0., 0., 0.11028503978328148, 0., 0., 0., 0., 0., 0., 0., 0., 0.9939, 0., 0., 0.]
n_qubits=4

alpha_j = Get_control_parameters(n_qubits, coef) 

state_circ_obj = State_Prep_Circuit(alpha_j)

circuit = (cirq.Circuit(cirq.decompose_once((state_circ_obj(*cirq.LineQubit.range(state_circ_obj.num_qubits()))))))


from quchem.Simulating_Quantum_Circuit import Get_wavefunction
state= Get_wavefunction(circuit,6)

In [ ]:
from quchem.Ansatz_Generator_Functions import *
xx = Ansatz(2,4)
# xx.Convert_occ_num_basis_to_basis_state([1,1,0,0])
xx.Convert_occ_num_basis_to_basis_state([0,0,1,1])

In [ ]:
n_shots=10000
VQE_exp = VQE_Experiment(QubitHam, circuit, n_shots)
VQE_exp.Calc_Energy()

In [ ]:
from quchem.quantum_circuit_functions import *
# |Ψ H2〉= 0.9939|0011〉−0.1106|1100〉
# |0011〉 = |3〉
# |1100〉= |12〉

coef = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
n_qubits=4

alpha_j = Get_control_parameters(n_qubits, coef) 

state_circ_obj = State_Prep_Circuit(alpha_j)

circuit = (cirq.Circuit(cirq.decompose_once((state_circ_obj(*cirq.LineQubit.range(state_circ_obj.num_qubits()))))))


from quchem.Simulating_Quantum_Circuit import Get_wavefunction
state= Get_wavefunction(circuit,6)

In [ ]:
xx.Convert_basis_state_to_occ_num_basis(np.array([1., 0., 0., 0, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))